In [1]:
import requests
import pandas as pd
import json
from json import JSONDecoder
from pprint import pprint
import time as T

In [2]:
# http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,87.38&date=2008-09-08&tp=1&format=json
# above is example

In [3]:
#import cleaned NFL schedule
schedule_csv = "NFL_Schedule.csv"

schedule = pd.read_csv(schedule_csv)
schedule.head()

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets
3,2008-09-07,1:02PM,1300,12,1,New England,Patriots,Patriots
4,2008-09-07,1:02PM,1300,12,1,Pittsburgh,Steelers,Steelers


In [4]:
#import cleaned NFL Stadium Locations
locations_csv = "NFL_Locations.csv"
locations = pd.read_csv(locations_csv)
#locations

In [5]:
locations["Latitude"] = locations["Latitude"].round(decimals=3)
locations["Longitude"] = locations["Longitude"].round(decimals=3)
#locations

In [6]:
#merge schedule with locations
merge = pd.merge(left=schedule, right=locations, how="left", left_on="City", right_on="Franchise")
merge.head()

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Franchise,Latitude,Longitude
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants,New York,40.43,-74.01
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles,Philadelphia,39.57,-75.07
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets,Miami,25.46,-80.12
3,2008-09-07,1:02PM,1300,12,1,New England,Patriots,Patriots,New England,42.21,-71.04
4,2008-09-07,1:02PM,1300,12,1,Pittsburgh,Steelers,Steelers,Pittsburgh,40.26,-80.00


In [7]:
clean_df = merge[["Full Date","Time","time_conv","time_index","Week","City","Home Team","Winner","Latitude","Longitude"]]
clean_df["Starting Game Temp (F)"] = ""
clean_df["weatherDesc"] = ""
clean_df["cloudcover"] = ""
clean_df

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Latitude,Longitude,Starting Game Temp (F),weatherDesc,cloudcover
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants,40.43,-74.01,,,
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles,39.57,-75.07,,,
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets,25.46,-80.12,,,
3,2008-09-07,1:02PM,1300,12,1,New England,Patriots,Patriots,42.21,-71.04,,,
4,2008-09-07,1:02PM,1300,12,1,Pittsburgh,Steelers,Steelers,40.26,-80.00,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,2015-01-17,1:06PM,1300,12,Division,Carolina,Panthers,Panthers,35.14,-80.50,,,
1733,2015-01-17,4:40PM,1600,15,Division,Denver,Broncos,Broncos,39.43,-105.01,,,
1734,2015-01-24,3:05PM,1500,14,ConfChamp,Denver,Broncos,Broncos,39.43,-105.01,,,
1735,2015-01-24,6:42PM,1800,17,ConfChamp,Carolina,Panthers,Panthers,35.14,-80.50,,,


In [10]:
for index, row in clean_df.iterrows():
    try:
        lat = row['Latitude']
        lng = row["Longitude"]
        date = row["Full Date"]
        location = f"{lat},{lng}"
        times = row["time_index"]
        print(location)
        print(date)
        print(times)
        
        T.sleep(.5)
    except IndexError:
        print("error")

40.43,-74.01
2008-09-04
18
39.57,-75.07
2008-09-07
12
25.46,-80.12
2008-09-07
12
42.21,-71.04
2008-09-07
12
40.26,-80.0
2008-09-07
12
42.54,-78.53
2008-09-07
12
36.09,-86.48
2008-09-07
12
39.17,-76.37
2008-09-07
12
32.43,-118.09
2008-09-07
15
41.3,-81.41
2008-09-07
15
37.48,-122.24
2008-09-07
15
39.46,-86.09
2008-09-07
19
44.3,-88.01
2008-09-08
18
37.47,-122.13
2008-09-08
21
38.38,-90.11
2008-09-14
12


KeyboardInterrupt: 

In [11]:
# set up table 1 df
table_1 = clean_df.iloc[:495,:]
table_1

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Latitude,Longitude,Starting Game Temp (F),weatherDesc,cloudcover
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants,40.43,-74.01,,,
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles,39.57,-75.07,,,
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets,25.46,-80.12,,,
3,2008-09-07,1:02PM,1300,12,1,New England,Patriots,Patriots,42.21,-71.04,,,
4,2008-09-07,1:02PM,1300,12,1,Pittsburgh,Steelers,Steelers,40.26,-80.00,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,2010-10-10,4:05PM,1600,15,5,Arizona,Cardinals,Cardinals,33.27,-112.05,,,
491,2010-10-10,4:15PM,1600,15,5,Oakland,Raiders,Raiders,37.47,-122.13,,,
492,2010-10-10,8:30PM,2000,19,5,San Francisco,49ers,Eagles,37.48,-122.24,,,
493,2010-10-11,9:15PM,2100,20,5,New York,Jets,Jets,40.43,-74.01,,,


In [ ]:
base_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9b6272a2d2e34f45bab144757210906&q="
for index, row in table_1.iterrows():
    try:
        lat = row['Latitude']
        lng = row["Longitude"]
        date = row["Full Date"]
        time = row["time_index"]
        location = f"{lat},{lng}"
        weather_test = f"{base_url},{location}&date={date}&tp=1&format=json"
        weather_info = requests.get(weather_test).json()
        table_1.loc[index, "Starting Game Temp (F)"] = weather_info['data']['weather'][0]['hourly'][time]['tempF']
        table_1.loc[index, "weatherDesc"] = weather_info['data']['weather'][0]['hourly'][time]['weatherDesc']
        table_1.loc[index, "cloudcover"]= weather_info['data']['weather'][0]['hourly'][time]['cloudcover']
        
        #print(table_1.iloc[[index],10:])
        
        T.sleep(1.5)
    except (KeyError,IndexError):
        print("error")

print("completed")

In [ ]:
table_1

In [ ]:
table_1.to_csv("Table 1.csv",encoding="utf-8",index=False)

In [ ]:
# run for table 2
table_2 = clean_df.iloc[495:990,:]

In [ ]:
base_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9b6272a2d2e34f45bab144757210906&q="
for index, row in table_2.iterrows():
    try:
        lat = row['Latitude']
        lng = row["Longitude"]
        date = row["Full Date"]
        time = row["time_index"]
        location = f"{lat},{lng}"
        weather_test = f"{base_url},{location}&date={date}&tp=1&format=json"
        weather_info = requests.get(weather_test).json()
        table_2.loc[index, "Starting Game Temp (F)"] = weather_info['data']['weather'][0]['hourly'][time]['tempF']
        table_2.loc[index, "weatherDesc"] = weather_info['data']['weather'][0]['hourly'][time]['weatherDesc']
        table_2.loc[index, "cloudcover"]= weather_info['data']['weather'][0]['hourly'][time]['cloudcover']
        
        #print(table_2.iloc[index])
        
        T.sleep(1.5)
    except (KeyError,IndexError):
        print("error")

print("completed")

In [ ]:
table_2

In [ ]:
table_2.to_csv("Table 2.csv",encoding="utf-8",index=False)

In [ ]:
table_3 = clean_df.iloc[990:1480,:]

In [ ]:
#run for table 3
table_3.head()

In [ ]:
base_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6cd0fcd19b014508966202712210906&q="
for index, row in table_3.iterrows():
    try:
        lat = row['Latitude']
        lng = row["Longitude"]
        date = row["Full Date"]
        time = row["time_index"]
        location = f"{lat},{lng}"
        weather_test = f"{base_url},{location}&date={date}&tp=1&format=json"
        weather_info = requests.get(weather_test).json()
        table_3.loc[index, "Starting Game Temp (F)"] = weather_info['data']['weather'][0]['hourly'][time]['tempF']
        table_3.loc[index, "weatherDesc"] = weather_info['data']['weather'][0]['hourly'][time]['weatherDesc']
        table_3.loc[index, "cloudcover"]= weather_info['data']['weather'][0]['hourly'][time]['cloudcover']
        
        #print(table_2.iloc[index])
        
        T.sleep(1)
    except (KeyError,IndexError):
        print("error")

print("completed")

In [ ]:
table_3

In [ ]:
table_3.to_csv("Table 3.csv",encoding="utf-8",index=False)

In [8]:
table_4 = clean_df.iloc[1480:,:]
table_4.head()

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Latitude,Longitude,Starting Game Temp (F),weatherDesc,cloudcover
1480,2014-12-14,4:05PM,1600,15,15,Tennessee,Titans,Jets,36.09,-86.48,,,
1481,2014-12-14,4:26PM,1600,15,15,Seattle,Seahawks,Seahawks,47.36,-122.20,,,
1482,2014-12-14,8:35PM,2000,19,15,Philadelphia,Eagles,Cowboys,39.57,-75.07,,,
1483,2014-12-15,8:30PM,2000,19,15,Chicago,Bears,Saints,41.53,-87.38,,,
1484,2014-12-18,8:26PM,2000,19,16,Jacksonville,Jaguars,Jaguars,30.20,-81.40,,,


In [ ]:
#http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,-87.38&date=2008-09-08&tp=1&format=json

In [ ]:
# done
table_1.head()
table_2.head()
table_3.head()

In [ ]:
table_4 = clean_df.iloc[495:975,:]
table_4.head()